For this homework, make sure that you format your notbook nicely and cite all sources in the appropriate sections. Programmatically generate or embed any figures or graphs that you need.

Names: __Lucas Calero Forero, Rebecca McBrayer__

Step 1: Word2Vec paper questions
---------------------------
1. Describe how a CBOW word embedding is generated.

A feedforward Neural Net without a hidden layer with a shared projection layer is used to classify the current word on every iteration based on its context (previous and future words). The output is a word embedding of size V.

2. What is a CBOW word embedding and how is it different from a skip-gram word embedding?

A CBOW word embedding is a grouping of vectorized representation of words. Its generated using a Continuous Bag of Words model that predicts and weights the current word based on its context. 
The biggest difference between the skip-gram model and CBOW model is the fact that skip-gram uses the current word as the input into the classifier to predict words within a range R of the word 1 < R < C where C is the maxiumum distance. This technique allows for weighted classification depending on the distance from w(i) and the current word. 

Their embeddings are quite similar in the end, one factor to consider between the two is the fact that skip-gram can be more accurate with less data.

3. What is the task that the authors use to evaluate the generated word embeddings?

The authors use the Semantic-Syntactic Word Relationship test set to evaluate their generated word embeddings, a test set developed by them for this paper and for future research.

4. What are PCA and t-SNE? Why are these important to the task of training and interpreting word embeddings?

PCA and t-SNE are methods of dimensionality reduction, projecting high-dimensional word embeddings into lower dimensional space. This is important foor training and interpreting word embeddings since embeddings can become extremely high dimensional, and projecting them into a lower space will both speed up training time and make it easier for humans to visualize and therefore interpret.

Step 2: Train your own word embeddings
--------------------------------

The Spooky Authors dataset is an old competition dataset from Kaggle. It has example sentences from famous works of writing from three different authors. For this assignment, each data point is a single sentence written by one of the three gothic authors.

Describe what data set you have chosen to compare and contrast with the Spooky Authors Dataset. Make sure to describe where it comes from and its general properties.

We chose the text of Les Miserables as our second dataset. This comes from Project Gutenburg and has over 500,000 words in its translated form.

In [1]:
# import your libraries here
import string
import urllib.request
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

from gensim.models import Word2Vec
from nltk.tokenize import sent_tokenize
import numpy as np
import pandas as pd
import tensorflow as tf

# Importing utility functions from Keras
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import SimpleRNN
from keras.layers import Embedding


In [2]:
# Variable Definitions
NGRAM = 3 # The size of the ngram language model you want to train

# The dimension of word embedding. 
# This variable will be used throughout the program
# you may vary this as you desire
EMBEDDINGS_SIZE = 200

### a) Train embeddings on GIVEN dataset

In [3]:
# Read the file 'spooky-author-identification/train.csv' 
spooky = pd.read_csv('train.csv')

In [4]:
# code to train your word embeddings
spooky_text = list(spooky['text'])

# Remove punctuation
exclude = set(string.punctuation)
spooky_nopun = []
for st in spooky_text:
    st = ''.join(ch for ch in st if ch not in exclude)
    spooky_nopun.append(st)

# Add start and ending tokens, and make all words lowercase
spooky_text = ["{} {} {}".format("".join(["<s> "] * (NGRAM - 1)),sentence.lower(),"".join(["</s> "] * (NGRAM-1))) for sentence in spooky_nopun]
spooky_data = [sentence.split() for sentence in spooky_text]
spooky_data = list(filter(lambda a: len(a)>(NGRAM-1)*2, spooky_data))  # Remove blank sentences


In [5]:
# Train the Word2Vec model from Gensim. 
# Below are the hyperparameters that are most relevant. 
# But feel free to explore other 
# options too:
# sg = 1
# window = 5
# size = EMBEDDING_SIZE
# min_count = 1

spooky_model = Word2Vec(sentences=spooky_data, size=EMBEDDINGS_SIZE, window=7, min_count=2, workers=4, sg=1)
spooky_model.save("word2vec.model")

In [6]:
# if you save your Word2Vec as the variable model, this will 
# print out the vocabulary size
print('Vocab size {}'.format(len(spooky_model.wv.vocab)))

Vocab size 15935


In [7]:
# You can save file in txt format, then load later if you wish.
spooky_model.wv.save_word2vec_format('embeddings.txt', binary=False)

### b) Train embedding on YOUR dataset

In [8]:
# Pull the data from Project Gutenburg
with urllib.request.urlopen("http://www.gutenberg.org/files/135/135-0.txt") as f:
    content = f.read()

text = content.decode('utf-8-sig')

# Remove parts of the text that aren't the book itself, plus extraneous whitespace
text = text[text.find("So long as there shall exist"):text.find("Publisher of the Italian translation ")]
# text = ''.join(ch for ch in text if ch not in ['\n','\r'])
text = text.replace('\n', ' ').replace('\r', ' ')

# Tokenize
sentences = sent_tokenize(text)

In [9]:
# Remove punctuation
exclude = set(string.punctuation)
text_nopun = []
for st in sentences:
    st = ''.join(ch for ch in st if ch not in exclude)
    text_nopun.append(st)

# Add start and ending tokens, and make all words lowercase
text = ["{} {} {}".format("".join(["<s> "] * (NGRAM-1)),sentence.lower(),"".join(["</s> "] * (NGRAM-1))) for sentence in text_nopun]
lesmis_data = [sentence.split() for sentence in text]
lesmis_data = list(filter(lambda a: len(a)>(NGRAM-1)*2, lesmis_data))  # Remove blank sentences


In [10]:
# Train the Word2Vec model from Gensim. 
# Below are the hyperparameters that are most relevant. 
# But feel free to explore other 
# options too:
# sg = 1
# window = 5
# size = EMBEDDING_SIZE
# min_count = 1

lesmis_model = Word2Vec(sentences=lesmis_data, size=EMBEDDINGS_SIZE, window=7, min_count=2, workers=4, sg=1)
lesmis_model.save("lesmis_word2vec.model")

In [11]:
# lesmis_model.wv.save_word2vec_format('lesmis_embeddings.txt', binary=False)
print('Vocab size {}'.format(len(lesmis_model.wv.vocab)))

Vocab size 15607


What text-normalization and pre-processing did you do and why? __We chose to do only very minimal text preprocessing, to keep our implementation simple. We removed punctuation and made all words lowercase, since those have no impact on the meaning of words (and therefore how they should be represented by an embedding. We added start and end tokens to make learning easier, but otherwise left the sentences alone. We did not lemmatize, as the different forms of words will have different meanings and therefore we decided to let them be embedded differently.__

Step 3: Evaluate the differences between the word embeddings
----------------------------

(make sure to include graphs, figures, and paragraphs with full sentences)

In [ ]:
spooky_model.wv.similar_by_word('king')

In [ ]:
lesmis_model.wv.similar_by_word('king')

In [ ]:
def display_closestwords_tsnescatterplot(model, word):
    
    arr = np.empty((0,200), dtype='f')
    word_labels = [word]

    # get close words
    close_words = model.wv.similar_by_word(word)
    
    # add the vector for each of the closest words to the array
    arr = np.append(arr, np.array([model.wv[word]]), axis=0)
    for wrd_score in close_words:
        wrd_vector = model.wv[wrd_score[0]]
        word_labels.append(wrd_score[0])
        arr = np.append(arr, np.array([wrd_vector]), axis=0)
        
    # find tsne coords for 2 dimensions
    tsne = TSNE(n_components=2, random_state=0)
    np.set_printoptions(suppress=True)
    Y = tsne.fit_transform(arr)

    x_coords = Y[:, 0]
    y_coords = Y[:, 1]
    # display scatter plot
    plt.scatter(x_coords, y_coords)

    for label, x, y in zip(word_labels, x_coords, y_coords):
        plt.annotate(label, xy=(x, y), xytext=(0, 0), textcoords='offset points')
    plt.xlim(x_coords.min()+0.00005, x_coords.max()+0.00005)
    plt.ylim(y_coords.min()+0.00005, y_coords.max()+0.00005)
    plt.show()

In [ ]:
display_closestwords_tsnescatterplot(spooky_model, 'relatives')

In [ ]:
result = lesmis_model.wv.most_similar(positive=['woman', 'king'], negative=['man'])
most_similar_key, similarity = result[0]  # look at the first match
print(f"{most_similar_key}: {similarity:.4f}")

In [ ]:
vis_data = TSNE(n_components=2, random_state=0).fit_transform(spooky_model.wv.vectors)

In [ ]:
vis_x = vis_data[:, 0]
vis_y = vis_data[:, 1]
plt.scatter(vis_x, vis_y, cmap=plt.cm.get_cmap("jet", 10))
plt.clim(-0.5, 9.5)
plt.show()

In [ ]:
vis_data = TSNE(n_components=2, random_state=0).fit_transform(lesmis_model.wv.vectors)

In [ ]:
vis_x = vis_data[:, 0]
vis_y = vis_data[:, 1]
plt.scatter(vis_x, vis_y, cmap=plt.cm.get_cmap("jet", 10))
plt.clim(-0.5, 9.5)
plt.show()

Cite your sources:
-------------

- https://medium.com/@aneesha/using-tsne-to-plot-a-subset-of-similar-words-from-word2vec-bb8eeaea6229
- https://indico.io/blog/visualizing-with-t-sne/

Step 4: Feedforward Neural Language Model
--------------------------

### a) First, encode  your text into integers

In [12]:
# Create word embedding dict to check if word exists in embedding (depending on if min_word>1)
spooky_embedding_dict = spooky_model.wv
lesmis_embedding_dict = lesmis_model.wv

In [13]:
flat_data = [word for sent in spooky_data for word in sent if word in spooky_embedding_dict]
spooky_vocab = list(set(flat_data))

# Initializing a Tokenizer
spooky_tokenizer = Tokenizer()
spooky_tokenizer.fit_on_texts(flat_data)

indexed_spooky_data = spooky_tokenizer.texts_to_sequences(spooky_data)
indexed_spooky_vocab = spooky_tokenizer.texts_to_sequences(spooky_vocab)

spooky_word_to_index = {word:index[0] for word, index in zip(list(spooky_vocab), indexed_spooky_vocab)}
spooky_index_to_word = {index[0]:word for word, index in zip(list(spooky_vocab), indexed_spooky_vocab)}

In [14]:
flat_data = [word for sent in lesmis_data for word in sent if word in lesmis_embedding_dict]
lesmis_vocab = list(set(flat_data))

# Initializing a Tokenizer
lesmis_tokenizer = Tokenizer()
lesmis_tokenizer.fit_on_texts(flat_data)

indexed_lesmis_data = lesmis_tokenizer.texts_to_sequences(lesmis_data)
indexed_lesmis_vocab = lesmis_tokenizer.texts_to_sequences(lesmis_vocab)

lesmis_word_to_index = {word:index[0] for word, index in zip(list(lesmis_vocab), indexed_lesmis_vocab)}
lesmis_index_to_word = {index[0]:word for word, index in zip(list(lesmis_vocab), indexed_lesmis_vocab)}

### b) Next, prepare your sequences from text

#### Fixed ngram based sequences (Used for Feedforward)

The training samples will be structured in the following format. 
Depending on which ngram model we choose, there will be (n-1) tokens 
in the input sequence (X) and we will need to predict the nth token (Y)

            X,						  y
    this,    process               however
    process, however               afforded
    however, afforded	           me

In [15]:
def generate_ngram_training_samples(content: list) -> list:
    '''
    Takes the encoded data (list of lists) and 
    generates the training samples out of it.
    Parameters:
    up to you, we've put in what we used
    but you can add/remove as needed
    return: 
    list of lists in the format [[x1, x2, ... , x(n-1), y], ...]
    '''
    ngrams = []
    for sent in content:
        for i in range(NGRAM-1, len(sent)):
            ngram = [sent[j] for j in range(i-(NGRAM-1), i+1)]
            ngrams.append(ngram)
    return ngrams

### c) Then, split the sequences into X and y and create a Data Generator

In [24]:
def read_embeddings(path: string, word_to_index: dict) -> (dict,dict):
    '''Loads and parses embeddings trained in earlier.
    Parameters and return values are up to you.
    '''
    # you may find generating the following two dicts useful:
    # word to embedding : {'the':1, ...}
    # index to embedding : {1:'the', ...} (inverse of word_2_embedding)
    
    model = Word2Vec.load(path)
    word_to_embedding = {}
    index_to_embedding = {}
    for word in model.wv.vocab.keys():
        index = word_to_index[word]
        embedding = model.wv[word]
        word_to_embedding[word] = embedding
        index_to_embedding[index] = embedding
    return word_to_embedding, index_to_embedding

### Create the x & y lists

In [27]:
def generate_encoded_input(source):
    ngram_source = generate_ngram_training_samples(source)
    encoded_X = []
    encoded_Y = []
    for ngram in ngram_source:
        encoded_X.append(ngram[:NGRAM-1])
        encoded_Y.append(ngram[-1])
        
    return encoded_X, encoded_Y

In [35]:
def data_generator(X: list, Y: list, vocab: list, num_sequences_per_batch: int, index_to_embedding: dict) -> (list,list):
    '''
    Returns data generator to be used by feed_forward
    https://wiki.python.org/moin/Generators
    https://realpython.com/introduction-to-python-generators/
    
    Yields batches of embeddings and labels to go with them.
    Use one hot vectors to encode the labels 
    (see the to_categorical function)
    
    '''
    num_sequences = 0
    
    while True:
        next_X = X[num_sequences: num_sequences+num_sequences_per_batch]
        next_X_vec = []
        for X_sent in next_X:
            X_sent_next = []
            for ii in X_sent:
                X_sent_next.extend(index_to_embedding[ii])
            next_X_vec.append(X_sent_next)
        
        next_Y = Y[num_sequences: num_sequences+num_sequences_per_batch]
        next_Y_cat = [tf.keras.utils.to_categorical(vec, num_classes=len(vocab)) for vec in next_Y]
        next_Y_tensor = np.array([tf.keras.backend.constant(cat) for cat in next_Y_cat])
        
        yield tf.keras.backend.constant(next_X_vec),next_Y_tensor
        num_sequences += num_sequences_per_batch

In [36]:
# initialize data_generator
def get_training_generator(source, vocab, encoded_X, encoded_Y, index_to_embedding):
    '''
    Sets parameters and collects data to build the data generator object
    '''
    sequences = source
    num_sequences_per_batch = 128 # this is the batch size
    steps_per_epoch = len(sequences)//num_sequences_per_batch  # Number of batches per epoch
    train_generator = data_generator(encoded_X, encoded_Y, vocab, num_sequences_per_batch, index_to_embedding)
    
    return train_generator, steps_per_epoch

# sample=next(train_generator) # this is how you get data out of generators
# sample[1].shape # (batch_size, (n-1)*EMBEDDING_SIZE)

### d) Train your models

In [20]:
# code to train a feedforward neural language model 
# on a set of given word embeddings

def create_model(model_size):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(64, input_dim=EMBEDDINGS_SIZE * (NGRAM-1)))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.Dense(model_size))
    model.add(tf.keras.layers.Activation('softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    print(model.summary())
    return model    

In [33]:
# Start training the model
def train_model(model, train_generator, steps_per_epoch):
    history = model.fit(x=train_generator,
              steps_per_epoch=steps_per_epoch,
              epochs=3)
    print(history.history['accuracy'])
    return history

### e) Generate Sentences

In [22]:
# generate a sequence from the model
def generate_seq(model: Sequential,
                 seed: list, 
                 n_words: int,
                 word_to_embedding: dict,
                 index_to_word: dict):
    '''
    Parameters:
        model: your neural network (trained)
        seed: [w1, w2, w(n-1)]
        n_words: generate a sentence of length n_words
        word_to_embedding: dictionary mapping words to their embeddings
        index_to_word: dictionary mapping indexes to the words they reference
    Returns: string sentence
    '''
    # initialize prediction list
    prediction_list = seed
    
    # Build the sentence
    while len(prediction_list) < n_words:
    
        # get last (NGRAM-1) words of list
        sample = prediction_list[-(NGRAM-1):]
    
        # Convert to embeddings, reshape properly
        sample_embedded = [word_to_embedding[word] for word in sample]
        sample_embedded = np.array(sample_embedded).reshape(-1, (NGRAM-1)*EMBEDDINGS_SIZE)
    
        # generate new word from these
        probabilities = model.predict(sample_embedded)[0]
        predicted_word_index = np.random.choice(np.arange(len(probabilities)), 1, p=probabilities)[0]
        # predicted_word_index = probabilities.argmax()
    
        # convert new word index to word and add it to list
        predicted_word = index_to_word[predicted_word_index]
        prediction_list.append(predicted_word)
        if predicted_word == "</s>":
            break
    
    return " ".join(prediction_list)

In [37]:
# Spooky Model
spooky_filepath = "word2vec.model"
SPOOKY_MODEL_SIZE = 15935

# Gather the input data
spooky_word_to_embedding, spooky_index_to_embedding = read_embeddings(spooky_filepath, spooky_word_to_index)
spooky_encoded_X, spooky_encoded_Y = generate_encoded_input(indexed_spooky_data)
spooky_data_generator, steps_per_epoch = get_training_generator(indexed_spooky_data,
                                                                spooky_vocab,
                                                                spooky_encoded_X, 
                                                                spooky_encoded_Y, 
                                                                spooky_index_to_embedding)

# Train the model
spooky_model = create_model(SPOOKY_MODEL_SIZE)
spooky_history = train_model(spooky_model, spooky_data_generator, steps_per_epoch)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 64)                25664     
_________________________________________________________________
activation_2 (Activation)    (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 15935)             1035775   
_________________________________________________________________
activation_3 (Activation)    (None, 15935)             0         
Total params: 1,061,439
Trainable params: 1,061,439
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
152/152 [==============================] - 29s 192ms/step - loss: 7.5522 - accuracy: 0.0611
Epoch 2/3
152/152 [==============================] - 28s 182ms/step - loss: 6.8755 - accuracy: 0.0866s - loss: 6 - ETA: 2s
Epoch 3/3
15

In [38]:
# Les Mis Model
lesmis_filepath = "lesmis_word2vec.model"
LESMIS_MODEL_SIZE = 15607

# gather the input data
lesmis_word_to_embedding, lesmis_index_to_embedding = read_embeddings(lesmis_filepath, lesmis_word_to_index)
lesmis_encoded_X, lesmis_encoded_Y = generate_encoded_input(indexed_lesmis_data)
lesmis_data_generator, steps_per_epoch = get_training_generator(indexed_lesmis_data,
                                                                lesmis_vocab,
                                                                lesmis_encoded_X, 
                                                                lesmis_encoded_Y, 
                                                                lesmis_index_to_embedding)

# Train the model
lesmis_model = create_model(LESMIS_MODEL_SIZE)
lesmis_history = train_model(lesmis_model, lesmis_data_generator, steps_per_epoch)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 64)                25664     
_________________________________________________________________
activation_4 (Activation)    (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 15607)             1014455   
_________________________________________________________________
activation_5 (Activation)    (None, 15607)             0         
Total params: 1,040,119
Trainable params: 1,040,119
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
229/229 [==============================] - 41s 178ms/step - loss: 7.1657 - accuracy: 0.0707
Epoch 2/3
229/229 [==============================] - 41s 178ms/step - loss: 6.7392 - accuracy: 0.0884s - loss: 6.7 - ETA: 1s - loss: 6.

In [41]:
# Spooky Sentences!
for _ in range(0,20):
    print(generate_seq(spooky_model, ["<s>", "<s>"], 20, spooky_word_to_embedding, spooky_index_to_word)[7:])

 and an precise negro it and the went characters being of your tink became bore realms size to
 as one march of energy is a zadok but with presence to the beheld of the loathed paralysed
 to poor removal nature river curiosity dark he i friends death expose reclaimed tumuli pinions scoundrelly perform side
 his abundantly hopes and oclock moslems it power and cases for to my politely repulsive christi connexion would
 hands to toward on the quietly of the passive reading did with burthen beheld a thoroughness inquisitive might
 the moment in been have previously and the general ladder of courtesy its mistaken and facial as expression
 with that not high its frame when man want and di forward and a complex although the had
 a revenge where uplands near to the seize in here scene me not received that he of a
 than the void notes gates in the my floated proceeded the may document his branches of preservation intercourse
 the knightly nothing he long strange up throat the colourless believ

In [42]:
# Les Mis Sentences!
for _ in range(20):
    print(generate_seq(lesmis_model, ["<s>", "<s>"], 20, lesmis_word_to_embedding, lesmis_index_to_word)[7:])

 is paper to fortune to to stepping one’s provençal life and darling round was ensued intervals the he
 would certain easily life” going was “there women this work of the marble according to strikes them police
 a earth is” pass see my monsieur wears what in said weather the plots of a torn as
 upon some moved which police were gazed sublime said a for jean time opened from into in the
 no bedroom who eyes will before m addressed the parish themselves the opposite penalty charming its attached his
 this respecting who stupor nothing there the woman of a mayor eyes good towards daybreak letting this “how
 done nor the on then to him but the director your yet me have been served think with
 would bound entering can her alms said the used is hour the all aurora bed of the his
 more accepted his booths god who again him that the wellbeloved silverware coteries the depths faults of them
 and was third the command was a or almost advanced been who since the pair which act would
 he had child

### f) Compare your generated sentences

*Compare these with sentences that could be produced using Shannon’s method with the statistical n-gram language models that we implemented earlier in the semester. Do your neural language models produce n-grams that were not observed during training? (1 paragraph, you may support this answer with code as desired).*

The sentences generated with feedforward network are more unique and varied than those generated using Shannon's method. Both the sentences here and those generated through Shannon's method make some level of grammatical sense, but at least with this data Shannon's method would perform better. Since the statistical way of generating sentences cannot create n-grams that did not occur in the training text, the Shannon's method sentences would appear more regular and contain more meaning. The sentences generated here include n-grams that were not observed during training, and there is little semantic sense contained in the majority of these sentences. The sentences generated from the Spooky Authors dataset are particularly nonsensical, and the sentences generated from Les Mis are ever so slightly more coherent. We assume this is because we had significantly more training data from Les Mis, and that if we trained these models with more information we would get increasingly comprehensible results. 

Sources Cited
----------------------------


https://keras.io/guides/sequential_model/